# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

In [8]:
import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI
import gradio as gr
import sqlite3

In [10]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'
openai = OpenAI()

DB = "wiedza.db"

API key looks good so far


In [11]:
# set up environment

OLLAMA_BASE_URL = "http://localhost:11434/v1"

ollama = OpenAI(base_url=OLLAMA_BASE_URL, api_key='ollama')

In [12]:
system_message = """
You are a technical assistant for residential construction knowledge.
Answer questions using the provided database and tools whenever factual information is required.
Give short, clear, and professional answers (maximum 5 sentences).
Do not guess or invent information.
If the database does not contain the answer, say explicitly that the information is not available.
When numerical values or technical rules are mentioned, they must come from the database.
"""

In [13]:
def get_information(query):
    print(f"DATABASE TOOL CALLED: Getting information for {query}", flush=True)
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()

        # Najpierw sprawdzamy komponenty
        cursor.execute(
            """
            SELECT name, manufacturer, category, notes
            FROM component
            WHERE name LIKE ? OR code = ?
            """,
            (f"%{query}%", query)
        )
        component = cursor.fetchone()

        if component:
            name, manufacturer, category, notes = component
            return (
                f"{name} "
                f"(category: {category}, manufacturer: {manufacturer}). "
                f"{notes}"
            )

        # Jeśli nie ma komponentu, szukamy wiedzy opisowej
        cursor.execute(
            """
            SELECT title, content
            FROM knowledge_chunk
            WHERE title LIKE ? OR content LIKE ?
            LIMIT 1
            """,
            (f"%{query}%", f"%{query}%")
        )
        chunk = cursor.fetchone()

    if chunk:
        title, content = chunk
        return f"{title}: {content}"

    return "No information available in the database."

In [14]:
get_information("F92-3-452")

DATABASE TOOL CALLED: Getting information for F92-3-452


'Wellenanker DBW 20 / Rd 20 GV (category: kotwa, manufacturer: Pfeifer). Standard dla połączenia ściany z fundamentem (parter)'

In [4]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [5]:
# Get gpt-4o-mini to answer, with streaming

def stream_answer(model, question):
    stream = openai.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are a helpful assistant who explains Python code clearly in Polish."},
            {"role": "user", "content": question},
    
          ],
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [6]:
stream_answer(MODEL_GPT, question)

Ten fragment kodu używa wyrażenia generatora z instrukcją `yield from`, aby zwrócić wartości z zestawu autorów książek, które są zdefiniowane w kolekcji `books`. Przyjrzyjmy się temu krok po kroku:

1. **Zestaw (`set`)**:
   - `{...}` to składnia do tworzenia zestawu (ang. set) w Pythonie. Zestaw to kolekcja unikalnych wartości. Oznacza to, że nawet jeśli w `books` pojawią się powtórzeni autorzy, w zestawie znajdziemy ich tylko raz.

2. **Wyrażenie za pomocą zrozumienia zbioru (`set comprehension`)**:
   - `book.get("author") for book in books if book.get("author")` to wyrażenie, które iteruje przez każdy element `book` w `books`. Używa metody `get` na słowniku `book`, aby uzyskać wartość przypisaną do klucza `"author"`.
   - Warunek `if book.get("author")` oznacza, że jeśli autor książki istnieje (tj. nie jest `None` ani pusty), to ta wartość jest dodawana do zestawu.

3. **Instrukcja `yield from`**:
   - `yield from` jest używane do zwracania wszystkich wartości z innego iteratora (w tym przypadku zestawu autorów). Umożliwia to efektywne "odsyłanie" wartości z jednego generatora do drugiego.

Podsumowując, cały ten kod tworzy zestaw autorów z kolekcji `books`, filtrując te książki, które mają przypisanego autora, a następnie zwraca każdy unikalny autor za pomocą `yield from`. 

Przykładowo, jeśli `books` zawiera:

```python
books = [
    {"title": "Book 1", "author": "Author A"},
    {"title": "Book 2", "author": "Author B"},
    {"title": "Book 3", "author": "Author A"},
    {"title": "Book 4", "author": None}
]
```

To wynik wyrażenia to zestaw `{"Author A", "Author B"}`. Włóżone do generatora, pozwala na iterowanie przez unikalnych autorów.

In [7]:
def stream_answer(model, question):
    stream = ollama.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are a helpful assistant who explains Python code clearly in Polish."},
            {"role": "user", "content": question},
    
          ],
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [ ]:
stream_answer(MODEL_LLAMA, question)